In [1]:
import numpy as np 
import ProjectPacks as pp
import matplotlib.pyplot as plt 
%matplotlib inline

**Confirmation of JHJ**

In [3]:
Amp,l,m,ut,vt,arrayHxpos = pp.formatSParams('Array_Profile.txt', 'Field_Profile.txt', td= 100)
SNum= len(Amp)
ArNum= len(ut) ## ut will always be a square mat

In [4]:
print(np.shape(ut))

(7, 7, 100)


In [13]:
# Analytic JHJ

Complete= np.zeros((3,3), dtype=complex)
Complete_An= np.zeros((3,3), dtype=complex)
for i in [21]: #in range(100):

    U= np.array(ut)[:,:,i]
    V= np.array(vt)[:,:,i]

    sumU,sqsumU= np.sum(np.triu(U,1)),np.sum(np.square(np.triu(U,1)))
    sumV, sqsumV= np.sum(np.triu(V,1)),np.sum(np.square(np.triu(V,1)))
    sumUV= np.sum(np.multiply(np.triu(U,1),np.triu(V,1)))

    ## Packing into Analytic JHJ
    JHJ= np.array([[3, -2j*np.pi*Amp[0]*sumU, -2j*np.pi*Amp[0]*sumV],
                  [2j*np.pi*Amp[0]*sumU, np.square(2*np.pi*Amp[0])*sqsumU, np.square(2*np.pi*Amp[0])*sumUV],
                  [2j*np.pi*Amp[0]*sumV, np.square(2*np.pi*Amp[0])*sumUV, np.square(2*np.pi*Amp[0])*sqsumV]])

    ## Calculating JHJ explicitly from J
    Psi= 2j*np.pi*(U*l[0]+V*m[0])
    expPsi= lambda unicoeff: np.exp(unicoeff*Psi)
    coeU= -2j*np.pi*Amp[0]*U
    coeV= -2j*np.pi*Amp[0]*V

    ## remember the -(minus) for negative exponents

    Jkt= np.array([expPsi(-1)[np.nonzero(np.triu(U,1))],
                   (coeU*expPsi(-1))[np.nonzero(np.triu(U,1))],(coeV*expPsi(-1))[np.nonzero(np.triu(V,1))]])
    
    

    ## NNNNBBBB!!!! with the above we need to be careful because indexing goes from left to right not up to down
    ## in the 3x3 case it doesnt matter but in the long run we need to make it more general

    J= Jkt.T # since there Jkt=Jktbar we don't need to concatenate anything
    Jh= J.conj().T ## because .H and .getH only works for matrices not ndarrays
    JhJ= np.dot(Jh,J) ## this is not element-wise, this is equivalent of matrix multiplication

    
    Complete += JhJ
    Complete_An += JHJ
    
   # print(np.allclose(JHJ, JhJ, rtol=1e-15, atol=1e-15, equal_nan=False))

In [14]:
print("\n Analytic JHJ: \n",Complete_An)
print("\n Computed JHJ: \n",Complete)
print("\n Analytic JHJinv: \n",np.linalg.pinv(Complete))
print('\n Analytic JHJ p(suedo) inv \n',np.linalg.pinv(JHJ).dot(JHJ))
print("\n Analytic JHJ = computed JHJ? \n",np.allclose(JHJ, JhJ, rtol=1e-15, atol=1e-15, equal_nan=False))


 Analytic JHJ: 
 [[  3.00000000e+00   +0.j           0.00000000e+00-2283.79638632j
    0.00000000e+00+2631.99335092j]
 [  0.00000000e+00+2283.79638632j   1.96394619e+06   +0.j
   -2.26337748e+06   +0.j        ]
 [  0.00000000e+00-2631.99335092j  -2.26337748e+06   +0.j
    2.60846129e+06   +0.j        ]]

 Computed JHJ: 
 [[  3.00000000e+00 +1.37679325e-17j  -1.86895064e-14 -2.28379639e+03j
    6.26270816e-14 +2.63199335e+03j]
 [ -1.86895064e-14 +2.28379639e+03j   1.96394619e+06 +2.63626018e-11j
   -2.26337748e+06 +9.86302280e-11j]
 [  6.26270816e-14 -2.63199335e+03j  -2.26337748e+06 -9.41917597e-11j
    2.60846129e+06 -5.93288496e-11j]]

 Analytic JHJinv: 
 [[  2.90476190e+00 +2.45607472e-16j  -9.20878586e-14 +1.45085161e-03j
   -7.99049411e-14 -1.67205440e-03j]
 [ -8.45407755e-13 -1.45085395e-03j   8.18600766e-07 -3.76263344e-16j
   -9.43407998e-07 +5.26548658e-16j]
 [ -7.33565737e-13 +1.67205236e-03j  -9.43405632e-07 -4.19404645e-16j
    1.08724113e-06 +3.76263681e-16j]]

 Analytic 

**Report**  
To a tolerance of 15 decimal places, JHJ is computed correctly, I am satisfied with this error. 